Server command:

```bash
# Setup
source ~/miniconda3/bin/activate
conda create --prefix ./envs python=3.10
pip install -e .
llama download --model-id Llama3.2-11B-Vision-Instruct
llama download --model-id Llama-Guard-3-1B

export LLAMA_STACK_PORT=5001
export INFERENCE_MODEL=meta-llama/Llama-3.2-11B-Vision-Instruct
export INFERENCE_PORT=8000
export SAFETY_MODEL=meta-llama/Llama-Guard-3-1B

# Start server
source ~/miniconda3/bin/activate
conda activate ./envs
llama stack build --template meta-reference-gpu --image-type conda && track llama stack run distributions/meta-reference-gpu/run-with-safety.yaml \
  --port 5001 \
  --env INFERENCE_MODEL=meta-llama/Llama-3.2-11B-Vision-Instruct
```

In [6]:
# Deps
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [8]:
import datasets

dataset = datasets.load_dataset("deepmind/narrativeqa")

dataset

/home/ubuntu/1xa100-2/llama-stack/envs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 32747
    })
    test: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 10557
    })
    validation: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 3461
    })
})

In [9]:
import pandas as pd

df = pd.DataFrame(dataset['validation'])
df['context'] = df['document'].apply(lambda x: x['text'])
df = df.rename(columns={'question': 'questions'})
df = df.groupby('context').agg({'questions': list}).reset_index()
df
df.head()

,context,questions
0,"<html>\n\n<head>\n<title>""Domino,"" by Richard ...",[{'text': 'Who planned the robbery that was be...
1,<html>\n<head><title>Airplane Script at IMSDb....,"[{'text': 'What is Ted Striker afraid of?', 't..."
2,<html>\n<head><title>All About Steve Script at...,"[{'text': 'What is Mary Horowitz's job?', 'tok..."
3,<html>\n<head><title>American Psycho Script at...,[{'text': 'Who is the first man Bateman muders...
4,"<html>\n<head><title>American, The Script at I...","[{'text': 'What does Jack do for a living?', '..."


In [10]:
import uuid
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.memory_insert_params import Document

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

providers = client.providers.list()
memory_banks_response = client.memory_banks.list()

bank_id = f"bank_{uuid.uuid4()}"
provider = providers["memory"][0]
client.memory_banks.register(
    memory_bank_id=bank_id,
    params={
        "embedding_model": "all-MiniLM-L6-v2",
        # Is the default for agent config: https://github.com/meta-llama/llama-stack/blob/66d8f4ffd126bff668434b314892a99fe854a034/llama_stack/providers/inline/agents/meta_reference/agent_instance.py#L668
        "chunk_size_in_tokens": 512,
    },
    provider_id=provider.provider_id,
)
bank_id

'bank_a7708879-1453-4d97-905d-368918ba428a'

In [11]:
documents = [
    Document(
        document_id=str(idx),
        content=context,
        mime_type="text/plain",
        metadata={},
    )
    for idx, context in zip(df.index, df["context"])
]

In [12]:
from tqdm import tqdm

for i in tqdm(range(len(documents))):
    client.memory.insert(
        bank_id=bank_id,
        documents=[documents[i]],
    )

  2%|█▍                                                                              | 2/115 [00:06<07:27,  3.96s/it]